# 7部
## 4章

### 4-3 単純パーセプトロン
### 4-4 活性化関数
(例)<br>
入力ベクトル : $x_1, x_2, 1(bias)$<br>
重み : $w_1, w_2, w_0$<br>
活性化関数(ステップ関数) : $h(x) = -1(x \leq 0), 1(x > 0)$<br>
式 : $y = h(w_0 \times 1 + w_1 \times x_1 + w_2 \times x_2)$

単純パーセプトロンは線形識別モデル(2値)<br>
ステップ関数は$h(X) = 0(x \leq 0), 1(x > 0)$とされてる場合もある，要は出力が2値になる<br>

活性化関数にはステップ関数の他にも，<br>
- ロジスティック関数　$h(x) = \frac{1}{1 + e^{-x}}$
    - 値域0-1の連続値を出力する($tanhみたいな形$)
- 恒等関数　$h(x) = x$
- ReLU関数　$h(x) = 0(x \leq 0), x(x > 0)$
    - $h(x)$を微分したら1なので勾配消失対策になる
    - 計算が早い
    - 0が生まれるので重要な特徴量のみ計算できるらしい(次元削減的な)
    
微分の話は<br>
>出力と目標ベクトルを比較し，損失が最小になるように重みが推定される

の部分の話になるがこの本ではスコープ外…誤差逆伝播法でググろう！

### 4-5 線形モデルからニューラルネットワークへ
単純パーセプトロンの活性化関数をロジスティック関数に，損失を交差エントロピー誤差にする

入力層はカウントしないはずなのでこの節の2層は1層の間違い(のはず)

### 4-7 ニューラルネットワーク
- 単純パーセプトロン(1層)
- 多層パーセプトロン/ニューラルネットワーク(2~層)
- ディープニューラルネットワーク/ディープネット(5層くらい~)

- パーセプトロンを増やす→全結合
- 畳み込み層を増やす→CNN
- 再帰層→RNN

この本では全結合のみスコープ

### 4-10 分析の準備

In [2]:
import numpy as np
import pandas as pd
import scipy as sp

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%precision 3

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


'%.3f'

### 4-11 データの読み込みと整形

In [11]:
iris = load_iris()
print(type(iris))
iris.feature_names

<class 'sklearn.utils.Bunch'>


['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [9]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [12]:
# 3種類中2種類の品種だけ使う, 4つの説明変数のうち2つだけ使う
X = iris.data[50:150, 0:2]
y = iris.target[50:150]

print(X.shape)
print(y.shape)

(100, 2)
(100,)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=98)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75, 2)
(75,)
(25, 2)
(25,)


train_test_split<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

3変数以上をsplitすることもできる<br>
`test_size=`でテストサイズを変えられる，デフォルト0.25

### 4-12 ロジスティック回帰

In [30]:
y_train

array([2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1,
       2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1,
       2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 1, 2])

In [31]:
X_train_df = pd.DataFrame(X_train, columns=["sepal_len", "sepal_wid"])
y_train_df = pd.DataFrame({"species" : y_train - 1})
iris_train_df = pd.concat([y_train_df, X_train_df], axis=1)

iris_train_df.head()

,species,sepal_len,sepal_wid
0,1,6.7,2.5
1,0,6.1,3.0
2,0,6.3,2.3
3,1,5.8,2.8
4,1,6.4,2.8


In [32]:
# 全ての変数を入れたモデル
logi_mod_full = smf.glm("species ~ sepal_len + sepal_wid", data=iris_train_df, family=sm.families.Binomial()).fit()

# 長さのみ
logi_mod_len = smf.glm("species ~ sepal_len", data=iris_train_df, family=sm.families.Binomial()).fit()

# 幅のみ
logi_mod_wid = smf.glm("species ~ sepal_wid", data=iris_train_df, family=sm.families.Binomial()).fit()

# Nullモデル
logi_mod_null = smf.glm("species ~ 1", data=iris_train_df, family=sm.families.Binomial()).fit()

# AICの比較
print("full", logi_mod_full.aic.round(3))
print("len", logi_mod_len.aic.round(3))
print("wid", logi_mod_wid.aic.round(3))
print("null", logi_mod_null.aic.round(3))

full 91.734
len 89.94
wid 101.003
null 105.852


In [33]:
# coef : 係数
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-11.3444,3.190,-3.556,0.000,-17.598,-5.091
sepal_len,1.8288,0.510,3.583,0.000,0.828,2.829


In [34]:
X_test_df = pd.DataFrame(X_test, columns=["sepal_len", "sepal_wid"])

logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)

# 正解数
true_train = sp.sum(logi_fit == (y_train - 1))
true_test = sp.sum(logi_pred == (y_test - 1))

# 正解率
print("train : ", true_train / len(y_train))
print("test : ", true_test / len(y_test))

train :  0.7333333333333333
test :  0.72


### 4-13 標準化
平均0，分散1にする

In [37]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train[0])
print(X_train_scaled[0])

[6.7 2.5]
[ 0.675 -1.08 ]


trainもtestも同じ変換器(scaler)で変換することでデータの対応が保持される

In [39]:
print(sp.std(X_train_scaled, axis=0))
print(sp.std(X_test_scaled, axis=0))

[1. 1.]
[1.103 0.908]


### 4-14 ニューラルネットワーク

In [44]:
nnet = MLPClassifier(hidden_layer_sizes=(100,100), alpha=0.07, max_iter=1000000, random_state=98)
nnet.fit(X_train_scaled, y_train)

print("train : ", nnet.score(X_train_scaled, y_train))
print("test : ", nnet.score(X_test_scaled, y_test))

train :  0.9066666666666666
test :  0.6


### 4-15 線形モデルの利点・NNの利点
- trainは高いがtestは低い．過学習．<br>
- NNは解釈が難しい．<br>
- なんでも深層学習はやめよう<br>

# 7部
## 5章

### 5-1 数理的な側面を学ぶ　
# 数学から逃げるな
~~逃げたい~~

### 5-2 古典的な「統計学入門」を学ぶ
松原ほか(1991)<br>
https://www.amazon.co.jp/統計学入門-基礎統計学Ⅰ-東京大学教養学部統計学教室/dp/4130420658

大学の数学の教科書という感じ

### 5-3 統計モデルを学ぶ
久保(2012)<br>
https://www.amazon.co.jp/データ解析のための統計モデリング入門――一般化線形モデル・階層ベイズモデル・MCMC-確率と情報の科学-久保-拓弥/dp/400006973X

- 本の最後の方で少しやった「線形モデル」の詳細+続きの話
- deviance(逸脱度)やAICもちゃんと解説してる
    - Nullモデルの説明もちゃんとある
    - Nullモデルはもっとも当てはまりが悪いモデルだからこれのdevianceから相対評価ができる…etc
- 次読む一冊に良さそう
    - まだ全部読めてない

### 5-4 機械学習を学ぶ
muller(2017)　→　Pythonではじめる機械学習<br>
https://www.amazon.co.jp/Pythonではじめる機械学習-―scikit-learnで学ぶ特徴量エンジニアリングと機械学習の基礎-Andreas-C-Muller/dp/4873117984

黄色い本は数学的な面が強いので　機械学習を学ぶ　とはまた違った感じ…全然読めてないけど

### 5-5 モデルの評価方法を学ぶ
- 機械学習は試行錯誤の繰り返し<br>
- クロスバリデーションなどで汎化誤差を見る

### 5-6 データサイエンス
- この本はデータサイエンスの基礎です